In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from keras.optimizers import SGD
from matplotlib.image import imread
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers.convolutional import Conv2D,MaxPooling2D, AveragePooling2D
from keras.layers import Dense, Flatten, Activation, Dropout
from collections import Counter
from shutil import copy

Using TensorFlow backend.


# Verzeichnisse für Train und Test Datensatz

In [2]:
train_dataset_path = 'DataSet/Eggs/dataset_train'
test_dataset_path = 'DataSet/Eggs/dataset_test'

In [3]:
train_dataset=ImageDataGenerator(rescale=1.0/255.0)
test_dataset=ImageDataGenerator(rescale=1.0/255.0)

In [ ]:
train_set=train_dataset.flow_from_directory(train_dataset_path, target_size=(64,64),
                                           batch_size=1, class_mode='categorical')
test_set=test_dataset.flow_from_directory(test_dataset_path, target_size=(64,64),
                                           batch_size=1, class_mode='categorical', shuffle = False)

batch_x, batch_y = train_set.next()
batch_x, batch_y = test_set.next()
print('Batch shape = %s, min = %.3f, max=%.3f' % (batch_x.shape, batch_x.min(), batch_x.max()))

In [ ]:
steps_per_epoch = len(train_set) 
validation_steps =len(test_set)
print('Training samples :', steps_per_epoch)
print('Test samples :', validation_steps)

In [ ]:
for i in range (0,300):
    train_batch_x, batch_y = train_set.next()
    image = train_batch_x[0]
    print(batch_y)
    plt.imshow(image)
    plt.show()
train_set.reset()
test_set.reset()

In [ ]:
for i in range (0,150):
    test_batch_x, batch_y = test_set.next()
    image = test_batch_x[0]
    print(batch_y)
    plt.imshow(image)
    plt.show()


# Topologie des neuronalen Netzes

In [ ]:
cnn_model = Sequential()

cnn_model.add(Conv2D(
    32, (3, 3), input_shape=(64, 64, 3), activation='relu',kernel_initializer='uniform', padding='same'))

cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Dropout(0.2))

cnn_model.add(Conv2D(
    32, (3, 3), activation='relu',kernel_initializer='uniform', padding='same'))

cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Dropout(0.2))

cnn_model.add(Flatten())

cnn_model.add(Dense(56, activation='relu', kernel_initializer='uniform'))

cnn_model.add(Dense(3, activation='softmax', kernel_initializer='uniform'))

In [ ]:
cnn_model.summary()

In [ ]:
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

## Anzahl der Gradientenabstiege

In [ ]:
epochs = 10

# Training

In [ ]:
train_set.reset()
test_set.reset()
history = cnn_model.fit_generator(train_set, steps_per_epoch=steps_per_epoch, validation_data=test_set,
                                      validation_steps=validation_steps, epochs=epochs)

# Genauigkeit auf Train und Test Datensatz

In [ ]:
# plot loss
plt.figure(figsize=(15,10))
plt.subplot(221)
plt.title('Loss')
plt.plot(history.history['loss'], color='blue', label='train')
plt.plot(history.history['val_loss'], color='red', label='test')

# plot accuracy
plt.subplot(222)
plt.title('Classification Accuracy')
plt.plot(history.history['accuracy'], color='blue', label='train')
plt.plot(history.history['val_accuracy'], color='red', label='test')
#plt.grid()
plt.show()

In [ ]:
_, acc = cnn_model.evaluate_generator(train_set, steps=len(train_set), verbose=3)
print('Accuracy on Training Data:  %.3f' % (acc * 100.0))
_, acc = cnn_model.evaluate_generator(test_set, steps=len(test_set), verbose=3)
print('Accuracy on Test Data :  %.3f' % (acc * 100.0))

In [ ]:
pred=cnn_model.predict_generator(test_set, 150, verbose=1)
predicted_class_indices=np.argmax(pred,axis=1)
labels = (test_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
filenames=test_set.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
results.to_csv("results.csv",index=False)


In [ ]:
for i in range (0,150):
    test_batch_x, batch_y = test_set.next()
    image = test_batch_x[0]
    print("Label:", batch_y, labels[np.argmax(batch_y,axis=1)[0]], "|", "Vorhersage:", "[" + "%.2f" % pred[i][0] + " " + "%.2f" % pred[i][1] + " " + "%.2f" % pred[i][2] + "]"  , predictions[i])
    plt.imshow(image)
    plt.show()